In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 2.1MB/s 
     |████████████████████████████████| 3.7MB 8.6MB/s 
     |████████████████████████████████| 1.0MB 33.3MB/s 
     |████████████████████████████████| 870kB 45.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=723f41ae73205a93d82b3ca3fcc6d23b1ca2a0650e69cd8446e232ee6587a6ff
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [0]:
import tensorflow as tf
import torch

import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from sklearn.svm import SVC
from sklearn import preprocessing

from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
from torch.utils.data import Dataset
from transformers import get_linear_schedule_with_warmup


from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from transformers import AdamW#, WarmupLinearSchedule

#from utils import (convert_examples_to_features,
                        #output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


Using TensorFlow backend.


In [0]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]


In [0]:


gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  gpuname=""
if torch.cuda.is_available():
  device=torch.device("cuda")
  n_gpu=torch.cuda.device_count()
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")




Found GPU at :/device:GPU:0
The device name is Tesla K80


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/task_b_distant.zip' 


In [0]:
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average','std']
taskb = pd.read_csv("task_b_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
#taskb=taskb[1:]

In [0]:
len(taskb)

188975

In [0]:
print(len(taskb[taskb.average >=0.5]))

39424


In [0]:
#len(englishdata1)
#39424 untargeted

In [0]:
print(len(taskb[taskb.average <0.5]))
#targeted 149551

149551


In [0]:

'''#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_b']]
englishdata=englishdata[1:]'''

'#GET THE DATA FROM THE PANDAS FRAME\nheaders=[\'id\',\'tweet\',\'subtask_a\',\'subtask_b\',\'subtask_c\']\nenglishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter=\'\t\',names=headers,low_memory=False)\nenglishdata=englishdata[[\'id\',\'tweet\',\'subtask_b\']]\nenglishdata=englishdata[1:]'

In [0]:
#englishdata = englishdata.dropna(subset=['subtask_b'])

In [0]:
#len(englishdata)

In [0]:
taskb=taskb[1:]

In [0]:
#englishdata.head()

In [0]:
'''englishtrain,englishtest= train_test_split(englishdata, test_size=0.2, random_state=42)
export_csv = englishtrain.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv', index = None, header=True)
print (englishtrain.head())
#englishtest,englishpredict= train_test_split(englishtemp, test_size=0.5, random_state=42)
export_csv = englishtest.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TestFileEnglish.tsv', index = None, header=True)
print (englishtest.head())'''
#export_csv = englishpredict.to_csv ('/content/drive/My Drive/EnglishData/predictFileEnglish.csv', index = None, header=True)
#print (englishpredict.head())

"englishtrain,englishtest= train_test_split(englishdata, test_size=0.2, random_state=42)\nexport_csv = englishtrain.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv', index = None, header=True)\nprint (englishtrain.head())\n#englishtest,englishpredict= train_test_split(englishtemp, test_size=0.5, random_state=42)\nexport_csv = englishtest.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TestFileEnglish.tsv', index = None, header=True)\nprint (englishtest.head())"

In [0]:
'''ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

def giveIds(sentence):
  maxlength=0
  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
  return maxlength'''

"ids_of_sentence=[]\nids_of_sentence_words=[]\nattention_masks=[]\n\ndef giveIds(sentence):\n  maxlength=0\n  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)\n  for t in sentence:\n      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)\n      if(maxlength<len(tokenized_sentence_id)):\n          maxlength=len(tokenized_sentence_id)\n  return maxlength"

In [0]:
#max_length=

In [0]:

def giveLabel(y):
  i=0
  y1=[]
  for r in y:
    if(y[i]>=0.5):
      y1.append(1)#1 means UNT 
    else:
      y1.append(0)
    i=i+1;
  return y1

headers=['id','text','average','std']

dfnumpy=taskb.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)
y1=giveLabel(y)

In [0]:
#preprocessedTweets=X[:,0]

In [0]:
x_train,x_test,y_train,y_test= train_test_split(x,y1, test_size=0.2, random_state=42)


In [0]:
from sklearn.utils import shuffle
x_train,y_train=shuffle(x_train,y_train)

In [0]:
x_train

array([['Trash as hell 🤣@USER Fuck it😂 if yo nigga don’t mean while gettin some head his ego too big and it’s trash'],
       ['@USER “How the fuck did you get that?”'],
       ['@1lilswoosh brooo i’m at the barbershop an that shit full blast 😒😒 🤦🏾\u200d♂️'],
       ...,
       ['My girl really stayed up until 5am watching OITNB 🤦\u200d♀️ I’m waking up to random sex scenes and shit lol'],
       ["That dude needs to remove the shit out of his ears in @USER because he can't keep his damn hands out of his pockets #LivePD"],
       ['FUCK it’s monday😞']], dtype=object)

In [0]:
'''x_train,y_train=readDataTrain()
x_test,y_test=readDataTest()
'''

'x_train,y_train=readDataTrain()\nx_test,y_test=readDataTest()\n'

In [0]:
#print(preprocessedTweets[0:5])

In [0]:
set(y_test)

{0, 1}

In [0]:
###IMPORTANT
'''xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'xtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [0]:
from transformers import RobertaForSequenceClassification as bfsc,AdamW,RobertaConfig
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

config = RobertaConfig.from_pretrained('roberta-base',output_attentions=False,output_hidden_states=False,num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)
model.cuda()

INFO:filelock:Lock 140440078735456 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpu_kvw3yy


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:filelock:Lock 140440078735456 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b1991

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json in cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:filelock:Lock 140438762409208 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:filelock:Lock 140440078833088 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggin

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt in cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:filelock:Lock 140440078833088 released on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d51427

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [0]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/robertabwoprenglish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/EnglishData/robertabwoprenglish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "beta","LayerNorm.weight","gamma"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=3e-5,eps=1e-8)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
#print(y_train.shape)
#print(y_test.shape)
#print(y_predict.shape)
z=le.fit_transform([0, 1])
yTrain=le.transform(y_train)
print(le.get_params())
print(yTrain.shape)
print(le.classes_)
yTest=le.transform(y_test)
print(le.classes_)

{}
(151179,)
[0 1]
[0 1]


In [0]:
yans1=["TIN" if yTrain[i]==0 else "UNT" for i in range(len(y_train))]


In [0]:
print(yans1.count("TIN"))
print(yans1.count("UNT"))

119777
31402


In [0]:
###IMPORTANT
'''
xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'\nxtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [0]:
#z=len(x_test)

In [0]:
#creating a dataset inspired from 
#https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784





In [0]:
#xytest[1]

In [0]:
#the customdataset section has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb  

In [0]:
MAXLENGTH=64

In [0]:
class EnglishTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        #attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        
        return x_train_pytorch,y_train_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytrain[0])
 

'''
z=0;
for batch_idx, data in enumerate(tdataloader): 
  if z==100:
    break;
  z=z+1;'''

'\nz=0;\nfor batch_idx, data in enumerate(tdataloader): \n  if z==100:\n    break;\n  z=z+1;'

In [0]:
class EnglishTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        #assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        #attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        #x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])



In [0]:
#xytrain=[x_train[:8000],y_train[:8000]]\
#MAXLENGTH=giveIds(x_train[0])
xytrain=[x_train,yTrain]
#tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tdataset = EnglishTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      

In [0]:

#tokenized_review = tokenizer.tokenize(str(xytest[0][0].flatten()))
 #y_test_pytorch=torch.tensor(y_test[0])
 #y_test[0]
 xytrain[0][1]

array(['@USER “How the fuck did you get that?”'], dtype=object)

In [0]:
#tokenized_review

In [0]:

#xytest=[x_test[:8000],y_test[:8000]]
xytest=[x_test,yTest]
#tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tedataset = EnglishTestDataset(xytest) 
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)
#trainData(tdataloader,tedataloader)

In [0]:

epochs=4
total_steps=len(tdataloader)*epochs
sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)



In [0]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    

    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "labels": batch[1]}
        model.zero_grad()     
        outputs = model(inputs["input_ids"],token_type_ids=None, labels=inputs["labels"])
        #print(type(outputs))
        loss, prediction_scores = outputs[:2]
        #print(loss)
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()
    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0
    
    for batch_idx, data in enumerate(tedataloader):
        
        batch = tuple(t.to(device) for t in data)            
        inputs = {"input_ids": batch[0], "labels": batch[1]}
        with torch.no_grad():        
           outputs = model(inputs["input_ids"],token_type_ids=None)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))
    torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/robertabwoprenglish.pth.tar')
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
      
    
print("ALL DONE!!!")

Streaming output truncated to the last 5000 lines.
labels Flattened [0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Predictions Argmax [0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0]
labels Flattened [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1]
Predictions Argmax [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
labels Flattened [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1]
Predictions Argmax [0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1]
labels Flattened [1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1]
Predictions Argmax [0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
labels Flattened [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
Predictions Argmax [0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
labels Flattened [0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Predictions Argmax [0 0 0 

In [0]:
print(type(outputs))

<class 'tuple'>


In [0]:
print(outputs[1])

IndexError: ignored

In [0]:
'''nlists=2
last=16000
size_of_itertrain=8000//nlists
size_of_itertest=8000//nlists
#print(size_of_itertrain)
for  i in range(nlists+1):
      
      end=(i+1)*size_of_itertrain
      if(last<end):
        end=last
      xytrain=[x_train[i*size_of_itertrain:end],y_train[i*size_of_itertrain:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tdataset = EnglishTrainDataset(xytrain)
      tsampler=RandomSampler(tdataset)
      tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      
      epochs=2
      total_steps=len(tdataloader)*epochs
      sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

      end=(i+1)*size_of_itertest
      if(last<end):
        end=last 
    
      xytest=[x_test[i*size_of_itertest:end],y_test[i*size_of_itertest:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tedataset = EnglishTestDataset(xytest)  
      tesampler=RandomSampler(tedataset)
      tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

      trainData(tdataloader,tedataloader)

      
y_test_pytorch=torch.tensor(xytest[1][index])'''

In [0]:
#y_test_pytorch=torch.tensor(xytest[1][40])

In [0]:
#len(xytest[1])

In [0]:
#len(xytest[0])

In [0]:
#len(xytrain[1])

In [0]:

MAXLENGTH

In [0]:

print("MAXLENGTH",MAXLENGTH)